In [0]:
!wget 'https://gist.githubusercontent.com/kroist/245ebe2e036541ecc4335ffb4c7d8bb7/raw/aea72fa14568b3c6dcbf28e721c324c855902a12/gistfile1.txt'

--2020-06-04 15:32:45--  https://gist.githubusercontent.com/kroist/594a0b23710a345358fa6a8a26ff8f27/raw/0d63887c4abe811c3df100a99f453a12ea14e0a5/gistfile1.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1111 (1.1K) [text/plain]
Saving to: ‘gistfile1.txt’

gistfile1.txt       100%[===================>]   1.08K  --.-KB/s    in 0s      

2020-06-04 15:32:45 (15.5 MB/s) - ‘gistfile1.txt’ saved [1111/1111]



In [0]:
f = open("gistfile1.txt","r")
# And for reading use
classes = f.readlines()
f.close()

In [0]:
classes = [c.replace('\n','').replace(' ','_') for c in classes]

In [0]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

In [0]:
!mkdir data

In [0]:
import urllib.request

def download():
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_', '%20')
    path = base + cls_url + '.npy'
    print(path)
    urllib.request.urlretrieve(path, 'data/'+c+'.npy')
  print('Download finished !')

In [0]:
download()

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/airplane.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/alarm%20clock.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/angel.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ant.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/anvil.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/arm.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/axe.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/backpack.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/barn.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/basket.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/basketball.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bit

In [0]:
def load_data(root, vfold_ratio=0.2, max_items_per_class=8000):
  all_files = glob.glob(os.path.join(root, '*.npy'))

  x = np.empty([0, 784])
  y = np.empty([0])
  class_names = []

  for idx, file in enumerate(all_files):
    data = np.load(file)
    data = data[0 : max_items_per_class, :]
    labels = np.full(data.shape[0], idx)

    x = np.concatenate((x, data), axis=0)

    y = np.append(y, labels)

    class_name, ext = os.path.splitext(os.path.basename(file))
    class_names.append(class_name)

  data = None
  labels = None


  permutation = np.random.permutation(y.shape[0])
  x = x[permutation, :]
  y = y[permutation]

  vfold_size = int(x.shape[0]/100*(vfold_ratio*100))


  x_test = x[0 : vfold_size, :]
  y_test = y[0 : vfold_size]

  x_train = x[vfold_size : x.shape[0], :]
  y_train = y[vfold_size : y.shape[0]]

  return x_train, y_train, x_test, y_test, class_names


In [0]:
x_train, y_train, x_test, y_test, class_names = load_data('data')
num_classes = len(class_names)
image_size = 28

In [0]:
print(len(x_train))
print(num_classes)

In [0]:
import matplotlib.pyplot as plt 
from random import randint
%matplotlib inline
idx = randint(0, len(x_train))
plt.imshow(x_train[idx].reshape(28, 28))
print(class_names[int(y_train[idx].item())])

In [0]:
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = keras.Sequential()

model.add(layers.Convolution2D(16, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))

model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

adam = tf.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics = ['top_k_categorical_accuracy'])

print(model.summary())


In [0]:
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

In [0]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Accuracy on tests: {:0.2f}%'.format(score[1] * 100))

In [0]:
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline
idx = randint(0, len(x_test))
img = x_test[idx]
plt.imshow(img.squeeze())
pred = model.predict(np.expand_dims(img, axis=0))[0]
ind = (-pred).argsort()[:5]
latex = [class_names[x] for x in ind]
print(latex)

In [0]:
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

In [0]:
model.save('keras.h5')

In [0]:
!mkdir model

In [0]:
!mv keras.h5 model/

In [0]:
!cp class_names.txt model/class_names.txt

In [0]:
!zip -r model.zip model

In [0]:
from google.colab import files
files.download('model.zip')